Combined code from both EOG and EMG

In [1]:
import os
import pandas as pd
from zipfile import ZipFile

import numpy as np
from scipy.signal import butter, filtfilt
import matplotlib.pyplot as plt

from scipy.stats import kurtosis, skew
from scipy.signal import welch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

Import function. This assumes app5_dataset.csv exists already. sleep_stage_lib\dataset.py creates this requirec file.

In [2]:
import pandas as pd

# Load raw signal data
raw_df = pd.read_csv("../data/processed/app5_dataset.csv")
print("Columns:", raw_df.columns)
print("First few rows:\n", raw_df.head())

# Group by file into a dictionary of DataFrames
raw_signals = {}

for filename, group in raw_df.groupby("file"):
    raw_signals[filename] = group.reset_index(drop=True)

print(f"\nLoaded {len(raw_signals)} files.")
print(f"Sample file: {list(raw_signals.keys())[0]}")
print(raw_signals[list(raw_signals.keys())[0]].head())


Columns: Index(['eog', 'emg', 'stage', 'set', 'file'], dtype='object')
First few rows:
     eog  emg  stage    set           file
0  39.0  3.0  awake  Train  0_subj_10.csv
1  36.0  4.0  awake  Train  0_subj_10.csv
2  36.0  2.0  awake  Train  0_subj_10.csv
3  40.0 -5.0  awake  Train  0_subj_10.csv
4  46.0 -2.0  awake  Train  0_subj_10.csv

Loaded 780 files.
Sample file: 0_subj_1.csv
   eog  emg   stage    set          file
0  0.0  5.0  nonrem  Train  0_subj_1.csv
1  5.0  7.0  nonrem  Train  0_subj_1.csv
2  7.0 -2.0  nonrem  Train  0_subj_1.csv
3  5.0 -5.0  nonrem  Train  0_subj_1.csv
4  2.0  2.0  nonrem  Train  0_subj_1.csv


Preprocessing for EOG and EMG

In [3]:
def bandpass_filter(data, lowcut, highcut, fs=200, order=4):
    nyq = 0.5 * fs
    if not (0 < lowcut < nyq and 0 < highcut < nyq):
        raise ValueError(f"Invalid cutoff frequencies: {lowcut}-{highcut} with fs={fs}")

    # Replace NaNs before filtering
    data = np.nan_to_num(data, nan=0.0)

    b, a = butter(order, [lowcut / nyq, highcut / nyq], btype='band')
    padlen = min(3 * (len(b) - 1), len(data) - 1)
    return filtfilt(b, a, data, padlen=padlen)


# ----------------------------------------
# Load raw signal data
# ----------------------------------------
raw_df = pd.read_csv("../data/processed/app5_dataset.csv")
print("Loaded raw_df with shape:", raw_df.shape)
print("Columns:", raw_df.columns)

# ----------------------------------------
# Apply filtering to EOG and EMG
# ----------------------------------------
filtered_list = []

for filename, group in raw_df.groupby("file"):
    group = group.copy()

    try:
        group["eog"] = bandpass_filter(group["eog"].values, lowcut=0.5, highcut=15.0)
        group["emg"] = bandpass_filter(group["emg"].values, lowcut=20.0, highcut=99.0)

        filtered_list.append(group)
    except Exception as e:
        print(f"Skipping {filename} due to filter error: {e}")

# Combine into full DataFrame
filtered_df = pd.concat(filtered_list, ignore_index=True)

print("\nFiltered dataset shape:", filtered_df.shape)
print("Sample:\n", filtered_df.head())


Loaded raw_df with shape: (9360780, 5)
Columns: Index(['eog', 'emg', 'stage', 'set', 'file'], dtype='object')

Filtered dataset shape: (9360780, 5)
Sample:
         eog       emg   stage    set          file
0  0.868383 -0.003382  nonrem  Train  0_subj_1.csv
1  2.599143  4.474229  nonrem  Train  0_subj_1.csv
2  4.096697 -2.229469  nonrem  Train  0_subj_1.csv
3  5.168638 -3.295843  nonrem  Train  0_subj_1.csv
4  5.685184  5.063718  nonrem  Train  0_subj_1.csv


In [4]:
def extract_features(signal, fs=200):
    # Time-domain features
    feat = {
        'mean': np.mean(signal),
        'std': np.std(signal),
        'min': np.min(signal),
        'max': np.max(signal),
        'median': np.median(signal),
        'energy': np.sum(signal ** 2),
        'kurtosis': kurtosis(signal),
        'skewness': skew(signal)
    }

    # Frequency-domain features using Welch's method
    freqs, psd = welch(signal, fs, nperseg=min(256, len(signal)))
    psd_norm = psd / (np.sum(psd) + 1e-10)  # Normalize PSD to sum to 1
    dominant_idx = np.argmax(psd)
    feat['dominant_frequency'] = freqs[dominant_idx]
    feat['spectral_entropy'] = -np.sum(psd_norm * np.log2(psd_norm + 1e-10))
    feat['spectral_bandwidth'] = np.sqrt(np.sum(((freqs - freqs[dominant_idx])**2) * psd_norm))

    return feat

In [5]:
# Extract features per file from your filtered_df (which contains columns 'file', 'stage', and cleaned 'eog'/'emg')
features = []
labels = []

for filename, group in filtered_df.groupby("file"):
    # Drop NaNs before filtering
    emg_values = group["emg"].dropna().values
    eog_values = group["eog"].dropna().values

    # Skip if there's not enough data to compute features
    if len(emg_values) < 10 or len(eog_values) < 10:
        print(f"Skipping {filename} (not enough valid data)")
        continue

    try:
        emg_feat = extract_features(emg_values)
        eog_feat = extract_features(eog_values)
    except Exception as e:
        print(f"Feature extraction failed for {filename}: {e}")
        continue

    row = {
        "file": filename,
        "set": group["set"].iloc[0],
        "stage": group["stage"].iloc[0],
    }

    # Prefix keys to avoid collision
    row.update({f"emg_{k}": v for k, v in emg_feat.items()})
    row.update({f"eog_{k}": v for k, v in eog_feat.items()})

    features.append(row)
    labels.append(group["stage"].iloc[0])  # Add stage to label list here

# Final feature DataFrame
features_df = pd.DataFrame(features)

# Sanity check
print("features_df shape:", features_df.shape)
print("labels length:", len(labels))
print("Label distribution:\n", pd.Series(labels).value_counts())


features_df shape: (780, 25)
labels length: 780
Label distribution:
 nonrem    297
awake     289
rem       194
Name: count, dtype: int64


In [6]:
# Encode labels and split the data
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

# Drop non-numeric columns before training
X = features_df.drop(columns=["file", "set", "stage"])
X_train, X_test, y_train, y_test = train_test_split(X, labels_encoded, test_size=0.3, random_state=42)

In [7]:
# Train a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [8]:
# Evaluate on the test set
predictions = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions, target_names=le.classes_))

Accuracy: 0.8205128205128205
              precision    recall  f1-score   support

       awake       0.90      0.79      0.84        99
      nonrem       0.72      0.90      0.80        73
         rem       0.87      0.77      0.82        62

    accuracy                           0.82       234
   macro avg       0.83      0.82      0.82       234
weighted avg       0.83      0.82      0.82       234

